In [1]:
import sys
sys.path.append("/kaggle/input/core.py/transformers/default/1")
print(sys.path)


['/kaggle/lib/kagglegym', '/kaggle/lib', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/root/.local/lib/python3.10/site-packages', '/opt/conda/lib/python3.10/site-packages', '/root/src/BigQuery_Helper', '/kaggle/input/core.py/transformers/default/1']


In [2]:
from core import CharacterTokenizer


In [3]:
!pip install transformers


In [3]:
import string

import sys
chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"

model_max_length = 64

tokenizer = CharacterTokenizer(chars, model_max_length)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
device

device(type='cuda')

In [5]:
example = "Привет"

tokens = tokenizer(example)

print(tokens)

{'input_ids': [0, 39, 42, 26, 12, 18, 46, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


Задание: обучите модель классификации букв для задачи расстановки ударения с помощью методов из библиотеки transformers. Датасет для обучения можно взять отсюда: https://github.com/Koziev/NLP_Datasets/blob/master/Stress/all_accents.zip



1. Напишите класс для Dataset/Dataloder и разбейте данные на случайные train / test сплиты в соотношении 50:50. (1 балл)

2. Попробуйте обучить одну или несколько из моделей: Bert, Albert, Deberta. Посчитайте метрику Accuracy на train и test. (1 балл). При преодолении порога в Accuracy на test 0.8: (+1 балл), 0.85: (+2 балла), 0.89: (+3 балла).

Пример конфигурации для deberta: https://huggingface.co/IlyaGusev/ru-word-stress-transformer/blob/main/config.json

In [6]:
import string
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, random_split
import torch
import torch.nn as nn

In [7]:
df = pd.read_table('/kaggle/input/dataset/all_accents.tsv', header=None, names = ['word', 'stressed_word'])
df['stress_idx'] = df['stressed_word'].str.find('^')

train_df, test_df = train_test_split(df, test_size=0.5)

In [8]:
train_df.head()


,word,stressed_word,stress_idx
52265,асессорский,ас^ессорский,2
523782,каноничная,канон^ичная,5
674701,михалко,мих^алко,3
246748,вышивальщицы,вышив^альщицы,5
605501,кюхельбекеры,кюхельб^екеры,7


In [9]:
class WordStressDataset(Dataset):
    def __init__(self, df, max_len):
        self.df = df
        self.max_len = max_len
        self.tokenizer = CharacterTokenizer(chars, model_max_length)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        word = self.df['word'].iloc[idx]
        stress_idx = self.df['stress_idx'].iloc[idx]

        tokens = self.tokenizer(
            word,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        labels = torch.zeros((self.max_len), dtype=torch.long)
        if stress_idx > 0:
            labels[stress_idx] = 1
        
        return tokens['input_ids'].flatten(), tokens['attention_mask'].flatten(), labels


train_dataset = WordStressDataset(train_df, model_max_length)
test_dataset = WordStressDataset(test_df, model_max_length)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
from transformers import DebertaV2ForTokenClassification, DebertaV2Config,  AdamW

config = DebertaV2Config(
    architectures="DebertaV2ForTokenClassification",
    model_type="deberta-v2",
    
    vocab_size=len(tokenizer.get_vocab()),  # Размер словаря (включая специальные токены)
    torch_dtype="float32",
    
    hidden_size=768,  # Размер скрытого слоя
    num_hidden_layers=4,  # Количество скрытых слоёв
    num_attention_heads=12,  # Количество голов внимания
    
    intermediate_size=1024,  # Размер промежуточного слоя
    hidden_act="gelu",  # Функция активации для скрытых слоёв
    hidden_dropout_prob=0.1,  # Вероятность dropout для скрытых слоёв
    attention_probs_dropout_prob=0.1,  # Вероятность dropout для вероятностей внимания
    max_position_embeddings=model_max_length,  # Максимальная длина последовательности
    #type_vocab_size=1,  # Количество типов токенов (для сегментации)
    #initializer_range=0.02,  # Диапазон инициализации весов
    #layer_norm_eps=1e-7,  # Точность нормализации слоёв
    #pad_token_id=0,  # ID токена для заполнения
    #position_embedding_type="absolute",  # Тип позиционного вложения
    #use_cache=True,  # Использовать кеш для ускорения вычислений
    
    num_labels=2,  # Количество классов для задачи классификации (0 - без ударения, 1 - с ударением)
)

config = DebertaV2Config(
    architectures="DebertaV2ForTokenClassification",
    model_type="deberta-v2",
    transformers_version="4.25.1",
    torch_dtype="float32",
    
    num_labels=2,

    hidden_size=1024,
    intermediate_size=2048,
    num_attention_heads=16,
    num_hidden_layers=5,
    
    hidden_dropout_prob=0.2,
    attention_probs_dropout_prob=0.2,
    position_biased_input=True,
    
    vocab_size=len(tokenizer.get_vocab()),
    
    max_length=model_max_length,
    max_position_embeddings=model_max_length,
    max_relative_positions=model_max_length,
    
    output_attentions=False,
    output_hidden_states=False,    
    
    #initializer_range=0.02,
    pooler_dropout = 0,
    pooler_hidden_act = "gelu",
    pooler_hidden_size = 1536
)

model = DebertaV2ForTokenClassification(config)
optimizer = torch.optim.AdamW(model.parameters(),lr = 1e-5,eps = 1e-8)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
from transformers import AlbertConfig, AlbertForTokenClassification

config = AlbertConfig(
    num_labels=2,
    hidden_size=600,
    intermediate_size=800,
    num_attention_heads=8,
    num_hidden_layers=4,
    hidden_dropout_prob=0.15,
    attention_probs_dropout_prob=0.15,
    max_position_embeddings=model_max_length,
    vocab_size=len(tokenizer.get_vocab()),
    output_attentions=False,
    output_hidden_states=False,
    pooler_dropout=0,
    pooler_hidden_act="gelu",
    pooler_hidden_size=736
)

model = AlbertForTokenClassification(config).to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr = 1e-5,eps = 1e-8)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [11]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
def accuracy(preds, labels):    
    return np.all(preds == labels, axis=1).sum() / len(labels)

def train(train_dataloader,epoch_i):
    total_train_loss = 0
    total_train_acc=0
    pbar = tqdm(train_dataloader, desc=f"Epoch {epoch_i}")
    for step, batch in enumerate(pbar):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.to(device)
        model.zero_grad()
        outputs = model(b_input_ids,attention_mask=b_input_mask,labels=b_labels)
        loss = outputs.loss
        total_train_loss += loss.item()
        logits = torch.argmax(outputs.logits.detach(), dim=2).cpu().numpy()
        label_ids = b_labels.cpu().numpy()
        total_train_acc += accuracy_score(logits, label_ids)
        loss.backward()
        optimizer.step()
    
    avg_train_loss = total_train_loss / len(train_dataloader)
    train_acc = total_train_acc/len(train_dataloader)
    print("epoch:", epoch_i)
    print("Average training loss: ",avg_train_loss)
    print("Train Accuracy: ", train_acc)
    return avg_train_loss, train_acc


def validate(val_dataloader):
    total_eval_accuracy = 0
    total_eval_loss = 0

    pbar = tqdm(val_dataloader, desc=f"Epoch {epoch_i}")
    for batch in pbar:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.to(device)
        with torch.no_grad():
            outputs = model(b_input_ids,attention_mask=b_input_mask,labels=b_labels)
        loss = outputs.loss
        total_eval_loss += loss.item()
        logits = torch.argmax(outputs.logits.detach(), dim=2).cpu().numpy()
        label_ids = b_labels.cpu().numpy()
        total_eval_accuracy += accuracy_score(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
    print("Validation Accuracy: ",avg_val_accuracy)
    avg_val_loss = total_eval_loss / len(val_dataloader)
    print("Validation Loss: ",avg_val_loss)
    
    return avg_val_loss, avg_val_accuracy

In [76]:
for epoch_i in range(0, 10):
    model.train()
    avg_train_loss = train(train_dataloader,epoch_i)
    
    
    model.eval()
    avg_test_loss, avg_test_accuracy = validate(test_dataloader)
    
print("test_loss",avg_test_loss)
print("test_accuracy",avg_test_accuracy)

Epoch 0: 100%|██████████| 13130/13130 [12:01<00:00, 18.21it/s]


epoch: 0
Average training loss:  0.024921558978375096
Train Accuracy:  0.4794677785086201


Epoch 0: 100%|██████████| 13130/13130 [08:02<00:00, 27.19it/s]


Validation Accuracy:  0.6856705382076669
Validation Loss:  0.017714026797937352


Epoch 1: 100%|██████████| 13130/13130 [12:00<00:00, 18.21it/s]


...

Validation Accuracy:  0.8433160383346028
Validation Loss:  0.009454715866822486


Epoch 9: 100%|██████████| 13130/13130 [12:00<00:00, 18.21it/s]


epoch: 9
Average training loss:  0.010659029457245735
Train Accuracy:  0.808812335560479


Epoch 9: 100%|██████████| 13130/13130 [08:03<00:00, 27.18it/s]

Validation Accuracy:  0.8513308422188374
Validation Loss:  0.009011224937846533
test_loss 0.009011224937846533
test_accuracy 0.8513308422188374


## Не хватило немного эпох до 0.89+ accuracy

In [77]:
for epoch_i in range(0, 4):
    model.train()
    avg_train_loss = train(train_dataloader,epoch_i)
    
    
    model.eval()
    avg_test_loss, avg_test_accuracy = validate(test_dataloader)
    
print("test_loss",avg_test_loss)
print("test_accuracy",avg_test_accuracy)

Epoch 0: 100%|██████████| 13130/13130 [12:01<00:00, 18.21it/s]


epoch: 0
Average training loss:  0.01020920947640964
Train Accuracy:  0.8179043264903414


Epoch 0: 100%|██████████| 13130/13130 [08:03<00:00, 27.17it/s]


Validation Accuracy:  0.8581683168316832
Validation Loss:  0.008526345587658971


Epoch 1: 100%|██████████| 13130/13130 [12:00<00:00, 18.22it/s]


...

Validation Accuracy:  0.8710042999492258
Validation Loss:  0.007873923504783262


Epoch 3: 100%|██████████| 13130/13130 [12:16<00:00, 17.83it/s]


epoch: 3
Average training loss:  0.009073447128750741
Train Accuracy:  0.8387408692792356


Epoch 3: 100%|██████████| 13130/13130 [08:42<00:00, 25.13it/s]

Validation Accuracy:  0.8744482260726073
Validation Loss:  0.007584021631514786
test_loss 0.007584021631514786
test_accuracy 0.8744482260726073


## Ну опять не вышло((

## Попробуем DEBERTA

In [12]:
from transformers import DebertaV2ForTokenClassification, DebertaV2Config,  AdamW



config = DebertaV2Config(

    architectures="DebertaV2ForTokenClassification",

    model_type="deberta-v2",

    vocab_size=len(tokenizer.get_vocab()),  # Размер словаря (включая специальные токены)

    torch_dtype="float32",


    hidden_size=768,  # Размер скрытого слоя

    num_hidden_layers=5,  # Количество скрытых слоёв

    num_attention_heads=12,  # Количество голов внимания


    intermediate_size=1024,  # Размер промежуточного слоя

    hidden_act="gelu",  # Функция активации для скрытых слоёв

    hidden_dropout_prob=0.15,  # Вероятность dropout для скрытых слоёв

    attention_probs_dropout_prob=0.15,  # Вероятность dropout для вероятностей внимания

    max_position_embeddings=model_max_length,  # Максимальная длина последовательности

    #type_vocab_size=1,  # Количество типов токенов (для сегментации)

    #initializer_range=0.02,  # Диапазон инициализации весов

    #layer_norm_eps=1e-7,  # Точность нормализации слоёв

    #pad_token_id=0,  # ID токена для заполнения

    #position_embedding_type="absolute",  # Тип позиционного вложения

    #use_cache=True,  # Использовать кеш для ускорения вычислений


    num_labels=2,  # Количество классов для задачи классификации (0 - без ударения, 1 - с ударением)

)



model = DebertaV2ForTokenClassification(config)

optimizer = torch.optim.AdamW(model.parameters(),lr = 1e-5,eps = 1e-8)

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
for epoch_i in range(0, 8):
    model.train()
    avg_train_loss = train(train_dataloader,epoch_i)
    
    
    model.eval()
    avg_test_loss, avg_test_accuracy = validate(test_dataloader)
    
print("test_loss",avg_test_loss)
print("test_accuracy",avg_test_accuracy)

Epoch 0: 100%|██████████| 13130/13130 [21:19<00:00, 10.26it/s]


epoch: 0
Average training loss:  0.020321631030820065
Train Accuracy:  0.6126946228276674


Epoch 0: 100%|██████████| 13130/13130 [10:53<00:00, 20.11it/s]


Validation Accuracy:  0.7536851040873318
Validation Loss:  0.01450563134055979


Epoch 1: 100%|██████████| 13130/13130 [21:19<00:00, 10.26it/s]


...

Validation Accuracy:  0.9016505616907845
Validation Loss:  0.006171510118922157


Epoch 7: 100%|██████████| 13130/13130 [21:18<00:00, 10.27it/s]


epoch: 7
Average training loss:  0.006987494059096417
Train Accuracy:  0.8847129664889566


Epoch 7: 100%|██████████| 13130/13130 [10:52<00:00, 20.12it/s]

Validation Accuracy:  0.9097788144199036
Validation Loss:  0.005747463491972981
test_loss 0.005747463491972981
test_accuracy 0.9097788144199036
